In [4]:
import re
import random
import heapq

lines = open('./data.txt', 'r').read().splitlines()
testlines = open('./testdata.txt', 'r').read().splitlines()

parser = re.compile(r'Blueprint (?P<bpnum>\d+): Each ore robot costs (?P<oreore>\d+) ore. Each clay robot costs (?P<clayore>\d+) ore. Each obsidian robot costs (?P<obsore>\d+) ore and (?P<obsclay>\d+) clay. Each geode robot costs (?P<geore>\d+) ore and (?P<geobs>\d+) obsidian.')

options = [
    "no_action",
    "ore",
    "clay",
    "obs",
    "geode",
]
vals = {
    "no_action": 0,
    "ore": 1,
    "clay": 2,
    "obs": 4,
    "geode": 8,
}

def max_possible_geodes(state, time):
    base = time * state.g_m
    add = ((time - 1) * time) / 2
    return base + add

class Miners:
    def __init__(self, ore=1, c=0, obs=0, g=0):
        (self.ore, self.c, self.obs, self.g) = (ore, c, obs, g)
    def copy(self):
        return Miners(self.ore, self.c, self.obs, self.g)
    def inc(self, type: str):
        if type == 'ore': self.ore += 1
        elif type == 'c': self.c += 1
        elif type == 'obs': self.obs += 1
        elif type == 'g': self.g += 1

class Resources:
    def __init__(self, ore, c, obs, g):
        (self.ore, self.c, self.obs, self.g) = (ore, c, obs, g)
    def __lt__(self, other):
        return self.ore < other.ore or self.c < other.c or self.obs < other.obs or self.g < other.g
    def __add__(self, other):
        return Resources(self.ore + other.ore, self.c + other.c, self.obs + other.obs, self.g + other.g)
    def __sub__(self, other):
        return Resources(self.ore - other.ore, self.c - other.c, self.obs - other.obs, self.g - other.g)

class BP:
    def __init__(self, line:str) -> None:
        result = parser.match(line)
        self.num = int(result['bpnum'])
        self.ore_ore = int(result['oreore'])
        self.clay_ore = int(result['clayore'])
        self.obs_ore = int(result['obsore'])
        self.obs_clay = int(result['obsclay'])
        self.geo_ore = int(result['geore'])
        self.geo_obs = int(result['geobs'])

class State:
    def __init__(self, ore=0, clay=0, obs=0, geo=0, ore_m=1, c_m=0, obs_m=0, g_m=0):
        self.ore = ore
        self.clay = clay
        self.obs = obs
        self.geo = geo
        self.ore_m = ore_m
        self.c_m = c_m
        self.obs_m = obs_m
        self.g_m = g_m
    def copy(self):
        return State(self.ore, self.clay, self.obs, self.geo, self.ore_m, self.c_m, self.obs_m, self.g_m)
    def potential(self):
        return self.g_m * 8 + self.obs_m * 4 + self.c_m * 2 + self.ore_m * 1
    
class Node:
    def __init__(self, time: int, state: State, decision: str, parent):
        self.time = time
        self.state = state
        self.decision = decision
        self.parent = parent
        self.next = []
    def __lt__(self, other):
        return self.potential() < other.potential()
    def __lte__(self, other):
        return self.potential() <= other.potential()
    def __gt__(self, other):
        return self.potential() > other.potential()
    def __gte__(self, other):
        return self.potential() >= other.potential()
    def add_next(self, next):
        self.next.append(next)
    def potential(self):
        return -(self.state.potential() + vals[self.decision]) * self.time

class PriorityQueue:
    def __init__(self, ):
        self.elements: list[Node] = []
    def empty(self) -> bool:
        return not self.elements
    def put(self, item: Node):
        heapq.heappush(self.elements, item)
    def get(self) -> Node:
        return heapq.heappop(self.elements)

class P1_Factory:
    def __init__(self, bp:BP) -> None:
        self.bp = bp
        self.max_geodes = 0
        self.tree = None
    
    def available_options(self, state: State):
        output = []
        for option in options:
            if option == 'no_action':
                output.append(option)
            elif option == 'ore':
                if state.ore >= self.bp.ore_ore:
                    output.append(option)
            elif option == 'clay':
                if state.ore >= self.bp.clay_ore:
                    output.append(option)
            elif option == 'obs':
                if state.ore >= self.bp.obs_ore and state.clay >= self.bp.obs_clay:
                    output.append(option)
            elif option == 'geode':
                if state.ore >= self.bp.geo_ore and state.obs >= self.bp.geo_obs:
                    output.append(option)
        return output
    
    def run_sim(self, node: Node):
        (time, state, decision) = (node.time, node.state, node.decision)
        if time == 0:
            self.max_geodes = max(self.max_geodes, state.geo)
            return True
        # elif max_possible_geodes(state, time) <= self.max_geodes:
        #     return
        
        if decision == 'ore':
            state.ore = state.ore - self.bp.ore_ore
        elif decision == 'clay':
            state.ore = state.ore - self.bp.clay_ore
        elif decision == 'obs':
            state.ore = state.ore - self.bp.obs_ore
            state.clay = state.clay - self.bp.obs_clay
        elif decision == 'geode':
            state.ore = state.ore - self.bp.geo_ore
            state.obs = state.obs - self.bp.geo_obs
        
        state.ore = state.ore + state.ore_m
        state.clay = state.clay + state.c_m
        state.obs = state.obs + state.obs_m
        state.geo = state.geo + state.g_m

        if decision == 'ore':
            state.ore_m = state.ore_m + 1
        elif decision == 'clay':
            state.c_m = state.c_m + 1
        elif decision == 'obs':
            state.obs_m = state.obs_m + 1
        elif decision == 'geode':
            state.g_m = state.g_m + 1

        options = self.available_options(state)
        if 'geode' in options:
            new_node = Node(time - 1, state.copy(), 'geode', node)
            self.run_sim(new_node)
        elif 'obs' in options:
            new_node = Node(time - 1, state.copy(), 'obs', node)
            self.run_sim(new_node)
        else:
            for o in options:
                new_node = Node(time - 1, state.copy(), o, node)
                self.run_sim(new_node)

    def start_sim(self):
        start_time = 24
        start_state = State()
        self.tree = Node(start_time, start_state, 'no_action', None)
        """ self.queue = PriorityQueue()
        self.queue.put(self.tree) """

        self.run_sim(self.tree)

        return self.max_geodes

bps = [BP(line) for line in testlines]

# factories = [P1_Factory(bp) for bp in bps]
# geodes = [f.start_sim() for f in factories]

In [9]:
bp = bps[0]

def get_resources_made_needed(miners: list[Miners], bp: BP):
    made:list[Resources] = []
    needed: list[Resources] = []
    for i in range(len(miners)):
        if i == 0:
            made.append(Resources(1, 0, 0, 0))
            needed.append(Resources(0, 0, 0, 0))
        else:
            mi = miners[i]
            lmi = miners[i - 1]
            lma = made[i - 1]
            lne = needed[i - 1]

            made.append(Resources(lma.ore + lmi.ore, lma.c + lmi.c, lma.obs + lmi.obs, lma.g + lmi.g))
            newNeeded = Resources(
                lne.ore + ((mi.ore - lmi.ore) * bp.ore_ore) + ((mi.c - lmi.c) * bp.clay_ore) + ((mi.obs - lmi.obs) * bp.obs_ore) + ((mi.g - lmi.g) * bp.geo_ore),
                lne.c + ((mi.obs - lmi.obs) * bp.obs_clay),
                lne.obs + ((mi.g - lmi.g) * bp.geo_obs),
                0
            )
            needed.append(newNeeded)
            if lma < newNeeded:
                raise Exception({
                    'minute': i,
                    'deficient': lma - newNeeded,
                    'made': made,
                    'needed': needed,
                })
    return made, needed

def copy_miners(m: list[Miners]):
    return [item.copy() for item in m]

def find_earliest(init_m: list[Miners], type='g', start=0, end=23):
    if start == end or start == end - 1:
        return init_m, None, None
    test_spot = (end - start) // 2
    test_m = copy_miners(init_m)
    [m.inc(type) for m in test_m[test_spot:]]
    try:
        made, needed = get_resources_made_needed(test_m, bp)
        # success, let's see if we can make it earlier
        return test_m, made, needed
    except Exception as e:
        d = e.args[0]['deficient']
        next_add = 'obs' if d.obs < 0 else 'c' if d.c < 0 else 'ore'
        return find_earliest(test_m, next_add, start, e.args[0]['minute'])

find_earliest([Miners() for i in range(24)])

KeyboardInterrupt: 